In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

df = pd.read_csv('CreditWorthiness - Sheet1.csv')
df.head()

,Cbal,Cdur,Chist,Cpur,Camt,Sbal,Edur,InRate,MSG,Oparties,...,Prop,age,inPlans,Htype,NumCred,JobType,Ndepend,telephone,foreign,creditScore
0,0 <= Rs. < 2000,9,all settled till now,Business,13790,Rs. < 1000,1 to 4 years,2,married or widowed male,no one,...,real estate,27,bank,own,1,employee with official position,1,yes,no,good
1,0 <= Rs. < 2000,15,dues not paid earlier,electronics,15250,no savings account,more than 7 years,4,single male,"yes, guarantor",...,real estate,50,none,own,2,employee with official position,1,yes,no,good
2,0 <= Rs. < 2000,36,none taken/all settled,Business,19410,Rs. < 1000,more than 7 years,4,single male,no one,...,Unknown,61,none,free,1,"employed either in management, self or in high...",1,yes,no,bad
3,0 <= Rs. < 2000,48,none taken/all settled,Business,144090,Rs. < 1000,1 to 4 years,2,single male,no one,...,Other cars etc.,25,none,own,1,employee with official position,1,yes,no,bad
4,no checking account,24,all settled till now,electronics,31690,Rs. < 1000,less than 1 year,4,divorced or separated or married female,no one,...,life insurance/building society,26,none,own,1,employee with official position,1,yes,no,good


In [2]:
# 1) Write a Python function that calculates the average age of customers who have settled their dues and have a credit score categorized as "good."

# Define the function to calculate average age of customers with settled dues and good credit score
def average_age_of_good_customers(data):
    good_customers = data[(data['creditScore'] == 'good') & (data['Chist'] != 'dues not paid earlier')]
    average_age = good_customers['age'].mean()
    return average_age

# Calculate and print the average age
average_age = average_age_of_good_customers(df)
print(f"Average age of customers with settled dues and good credit score: {average_age:.2f}" )

Average age of customers with settled dues and good credit score: 34.56


In [3]:
# 2) Write a Python function that determines the percentage of customers who have a savings account and own more than one vehicle.

def percentage_of_customers(data):
    savings_and_more_vehicles = data[(df['Sbal'] != 'no savings account') & (data['Prop'] == 'Other cars etc.')]
    percentage = (len(savings_and_more_vehicles) / len(df)) * 100
    return percentage

percentage = percentage_of_customers(df)
print(f"Percentage of customers with savings account and more than one vehicle: {percentage}%")

Percentage of customers with savings account and more than one vehicle: 27.6%


In [4]:
# 3) Write a Python function that identifies the most common job type among customers who have taken credit for a new vehicle.

def most_common_job_for_new_vehicle(data):
    new_vehicle_data = data[data['Cpur'] == 'new vehicle']
    most_common_job = new_vehicle_data['JobType'].value_counts().idxmax()
    return most_common_job

most_common_job = most_common_job_for_new_vehicle(df)
print(f"Most common job type for customers with new vehicle credit: {most_common_job}")

Most common job type for customers with new vehicle credit: employee with official position


In [5]:
# 4) Write a Python function that calculates the median value of the duration of credit (Cdur) for customers who are divorced, separated, or married females.

def median_duration_for_married_females(data):
    married_females_data = data[data['MSG'] == 'divorced or separated or married female']
    median = married_females_data['Cdur'].median()
    return median

median = median_duration_for_married_females(df)
print(f"Median duration of credit for married females: {median}")

Median duration of credit for married females: 18.0


In [6]:
df['Camt'].describe()

count      1000.00000
mean      32592.58000
std       28227.36876
min        2380.00000
25%       13535.00000
50%       23075.00000
75%       39602.50000
max      184120.00000
Name: Camt, dtype: float64

In [7]:
# 5) Write a Python function that categorizes customers based on their credit amount (Camt) into bins: 'low', 'medium', and 'high', and returns the count
# of customers in each category.


def categorize_credit_amount(data):
    quartiles = data['Camt'].quantile([0.25, 0.5, 0.75, 1.0])
    bin_edges = [quartiles.iloc[0], quartiles.iloc[1], quartiles.iloc[2], quartiles.iloc[3]]
    bin_labels = ['low', 'medium', 'high']
    data['CreditCategory'] = pd.cut(data['Camt'], bins=bin_edges, labels=bin_labels)
    
    # Count the number of customers in each category
    category_counts = data['CreditCategory'].value_counts()
    
    return category_counts

categorize_credit_amount(df)

low       250
medium    250
high      250
Name: CreditCategory, dtype: int64

In [8]:
df.drop(columns='CreditCategory',inplace=True)

In [9]:
# 6) Calculate the conditional probability that a customer has a Credit Amount (Camt) greater than or equal to 30000, given that they own their house.

owns_house_count = len(df[df['Htype'] == 'own'])
high_camt_and_own_count = len(df[(df['Camt'] >= 30000) & (df['Htype'] == 'own')])

conditional_probability = high_camt_and_own_count / owns_house_count

print(f"Conditional Probability: {conditional_probability:.2f}")

Conditional Probability: 0.35


In [10]:
# 7) Perform a hypothesis test to determine whether there is a significant difference in the average credit scores between
# customers who own their house and customers who do not own their house. Use a significance level of 0.05.
# Please note that you would need to follow the steps of hypothesis testing, such as setting up null and alternative hypotheses,
# calculating the test statistic, and comparing it to the critical value or p-value to make a conclusion about the significance 
# of the difference in average credit scores.

# Null hypothesis is the average credit is same for both the groups
# as the size is more so ztest is used for hypothesis testing

import scipy.stats as stats
from statsmodels.stats.weightstats import ztest


# Separate the data into two groups: customers who own their house and customers who do not own their house
own_house_scores = df[df['Htype'] == 'own']['Camt']
not_own_house_scores = df[df['Htype'] != 'own']['Camt']

# Calculate the mean of credit scores for both groups
own_house_mean = own_house_scores.mean()
not_own_house_mean = not_own_house_scores.mean()

# Perform two-sample t-test
z_statistic, p_value = ztest(own_house_scores, not_own_house_scores)

# Compare p-value with significance level
alpha = 0.05
if p_value < alpha:
    conclusion = "Reject null hypothesis: There is a significant difference in average credit."
else:
    conclusion = "Fail to reject null hypothesis: There is no significant difference in average credit."

print("z-statistic:", z_statistic)
print("P-value:", p_value)
print(conclusion)

z-statistic: -3.737763012772279
P-value: 0.00018566482526763926
Reject null hypothesis: There is a significant difference in average credit.


In [11]:
# 8) In the context of the given data set with customer information and credit scores, 
# apply a feature selection technique to identify the most important features that contribute 
# to predicting credit scores accurately. Explain the rationale behind your choice of feature 
# selection technique and discuss the impact of feature reduction on model performance.

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.ensemble import RandomForestClassifier

# Selecting the numeric columns and categorical columns for feature transformation
numeric_columns = df.select_dtypes(include=['int64', 'float64']).columns
categorical_columns = df.select_dtypes(include=['object', 'category']).columns

# Exclude the target variable 'creditScore'
numeric_columns = [col for col in numeric_columns if col != 'creditScore']
categorical_columns = [col for col in categorical_columns if col != 'creditScore']

# Separate the features and the target variable
X_numeric = df[numeric_columns]
X_categorical = df[categorical_columns]
y = df['creditScore']

# Convert y labels to numeric values using LabelEncoder
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Standardize the numeric features
scaler = StandardScaler()
X_numeric_scaled = scaler.fit_transform(X_numeric)

# Apply label encoding to categorical features
X_categorical_encoded = X_categorical.copy()
for col in categorical_columns:
    le = LabelEncoder()
    X_categorical_encoded[col] = le.fit_transform(X_categorical[col])

# Combine the scaled numeric features and the label encoded categorical features
X_combined = pd.concat([pd.DataFrame(X_numeric_scaled, columns=numeric_columns), X_categorical_encoded], axis=1)

# Perform PCA
pca = PCA()
X_pca = pca.fit_transform(X_combined)

# Explained variance ratio of each component
explained_variance_ratio = pca.explained_variance_ratio_

# Create a DataFrame to store column names and their explained variance ratios
explained_variance_df = pd.DataFrame({'Column': X_combined.columns, 'Explained Variance Ratio': explained_variance_ratio})

# Sort the DataFrame by explained variance ratio in descending order
explained_variance_df = explained_variance_df.sort_values(by='Explained Variance Ratio', ascending=False)

print(explained_variance_df)

# columns with high Explained Variance Ratio is a important feature that contribute to predicting credit score

       Column  Explained Variance Ratio
0        Cdur                  0.329357
1        Camt                  0.099177
2      InRate                  0.084314
3         age                  0.069439
4     NumCred                  0.056035
5     Ndepend                  0.052260
6        Cbal                  0.049098
7       Chist                  0.044746
8        Cpur                  0.039477
9        Sbal                  0.038156
10       Edur                  0.030080
11        MSG                  0.027498
12   Oparties                  0.026114
13       Rdur                  0.011059
14       Prop                  0.010681
15    inPlans                  0.009201
16      Htype                  0.008349
17    JobType                  0.007047
18  telephone                  0.006632
19    foreign                  0.001280


In [12]:
# 9) Utilizing the data set containing customer features and credit scores, create a classification model 
# to classify customers into two categories: "good" credit score and "bad" credit score. Train the model 
# and assess its accuracy, precision, recall, and F1-score evaluation will be done on these parameters.

# Define the function
def perfrom_pca(n):
    X_train, X_test, y_train, y_test = train_test_split(X_pca, y_encoded, test_size=0.2, random_state=0)
    pca = PCA(n_components=n)
    pca_x_train = pca.fit_transform(X_train)
    pca_x_test = pca.transform(X_test)
    classifier = RandomForestClassifier(max_depth=2, random_state=0)
    classifier.fit(pca_x_train, y_train)
    y_pred = classifier.predict(pca_x_test)

    # Evaluate the model performance
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred )
    recall = recall_score(y_test, y_pred )
    f1 = f1_score(y_test, y_pred)

    print(f"Accuracy: {accuracy:.2f}")
    print(f"Precision: {precision:.2f}")
    print(f"Recall: {recall:.2f}")
    print(f"F1-score: {f1:.2f}")
    print('\n')

for x in range(1, 5): perfrom_pca(x)

Accuracy: 0.68
Precision: 0.68
Recall: 0.99
F1-score: 0.81


Accuracy: 0.69
Precision: 0.69
Recall: 1.00
F1-score: 0.81


Accuracy: 0.69
Precision: 0.69
Recall: 1.00
F1-score: 0.81


Accuracy: 0.69
Precision: 0.69
Recall: 1.00
F1-score: 0.82




In [13]:
# 10) How many Persons are there who are in their 50s and whose Marital status is Single

# Filter the DataFrame based on age and marital status
persons_in_50s_single = df[(df['age'] >= 50) & (df['age'] < 60) & (df['MSG'] == 'single male')]

# Get the count of persons
count_persons_in_50s_single = len(persons_in_50s_single)

print(f"Number of persons in their 50s and single: {count_persons_in_50s_single}")


Number of persons in their 50s and single: 46


In [14]:
# 11) Create a UDF that will extract the data in table form based on credit score(credit score) and Purpose of the loan(Cpur)

#below is the sql query table name is Credit_Worthiness

# CREATE FUNCTION ExtractData(
#     @creditScore NVARCHAR(50),
#     @Cpur NVARCHAR(50)
# )
# RETURNS TABLE
# AS
# RETURN
# (
#     SELECT *
#     FROM Credit_Worthiness
#     WHERE creditScore = @creditScore AND Cpur = @Cpur
# );


In [15]:
# 12) Write a query to fetch the data of the borrower who is foreign and has a duration of credit of more than 10 months.


# SELECT *
# FROM Credit_Worthiness
# WHERE foreign = 'yes' AND Cdur > 10;

In [16]:
# 13) Write a query to fetch the data whose Balance in the savings bank is greater than 1000 RS but still have a Bad credit score.

# SELECT *
# FROM Credit_Worthiness
# WHERE Sbal NOT IN ('Rs. < 1000', 'no savings account')
# AND creditScore = 'bad';

In [17]:
# 14) Based on every job type find the highest amount under credit/loan and find the entire data for that borrower

# SELECT *
# FROM Credit_Worthiness
# WHERE (JobType, Camt) IN (
#     SELECT JobType, MAX(Camt) AS MaxAmount
#     FROM Credit_Worthiness
#     GROUP BY JobType
# )

